In [ ]:
from dotenv import load_dotenv

load_dotenv("../../config/.env")

True

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-chat", model_provider="deepseek")

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 通过一次大模型调用总结

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "请对以下内容写一个简明的总结：\\n\\n{context}")]
)
chain = create_stuff_documents_chain(llm, prompt)

for token in chain.invoke({"context": docs}):
    print(token, end="", flush=True)

这篇博客文章《LLM Powered Autonomous Agents》由Lilian Weng撰写，系统地概述了如何利用大型语言模型（LLM）作为核心控制器来构建自主智能体系统。文章将这类系统分解为三个核心组件，并结合实际案例和挑战进行了深入探讨。

### 核心摘要：

**1. 代理系统概述**
一个LLM驱动的自主代理系统以LLM作为“大脑”，并辅以三个关键组件：
- **规划**：将复杂任务分解为子目标，并进行自我反思以改进策略。
- **记忆**：包括短期记忆（利用模型的上下文学习能力）和长期记忆（通过外部向量存储实现信息的持久化和快速检索）。
- **工具使用**：学习调用外部API来获取模型权重中缺失的信息（如实时数据、代码执行能力）。

**2. 核心组件详解**

- **规划**
    - **任务分解**：通过思维链、思维树等提示技术，或将规划外包给经典规划器（如LLM+P），将大任务拆解为小步骤。
    - **自我反思**：通过ReAct、Reflexion、Chain of Hindsight和Algorithm Distillation等框架，使代理能够从过去的错误中学习，迭代改进其输出和策略。

- **记忆**
    - **类型**：类比人类记忆，分为感觉记忆（嵌入表示）、短期记忆（上下文学习）和长期记忆（外部向量存储）。
    - **检索**：使用近似最近邻算法（如HNSW, FAISS, ScaNN）进行快速最大内积搜索，以实现高效的内存检索。

- **工具使用**
    - 代理通过学习调用外部工具（如计算器、搜索引擎、API）来扩展其能力。
    - 相关框架包括MRKL、TALM、Toolformer，以及实践中的ChatGPT插件和OpenAI函数调用。
    - **HuggingGPT** 和 **API-Bank** 是 benchmarks 和框架，展示了LLM如何作为任务规划器，协调和利用外部模型与工具。

**3. 案例研究**

- **科学发现代理**：如ChemCrow，通过集成13个专家设计的工具，在化学合成等领域完成任务。研究表明，尽管LLM自我评估可能相近，但专家评估显示工具增强的代理表现更优。
- **生成式代理模拟**：在一个沙盒环境中模拟25个由LLM驱动的虚拟角色的

### 通过Map-Reduce架构并行化总结

In [10]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=0,
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [11]:
import operator
from typing import List, Annotated, TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langchain.chains.combine_documents.reduce import (
    acollapse_docs,
    split_list_of_docs,
)

# 单次处理的最大token数量
token_max = 1000

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "写下以下内容的简明摘要：{context}")]
)

reduce_template = """
以下是一组摘要：
{docs}
请将这些内容提炼成一个最终的、综合的摘要，概括主要主题。
"""
reduce_prompt = ChatPromptTemplate(
    [("human", reduce_template)]
)

# 主图状态：管理整个摘要流程的数据流
class OverallState(TypedDict):
    contents: List[str]
    summaries: Annotated[list, operator.add]
    collapsed_summaries: List[Document]
    final_summary: str

class SummaryState(TypedDict):
    content: str

# 计算文档列表的总token数量
def length_function(documents: List[Document]) -> int:
    return sum(llm.get_num_tokens(doc.page_content) for doc in documents)

# 为每个文档内容创建一个Send对象，实现并行处理
def map_summaries(state: OverallState):
    return [
        Send("generate_summary", {"content": content}) for content in state["contents"]
    ]

# 将多个摘要合并成一个
async def _reduce(input: dict) -> str:
    prompt = reduce_prompt.invoke(input)
    response = await llm.ainvoke(prompt)
    return response.content

# 判断是否需要继续合并摘要
def should_collapse(state: OverallState):
    num_tokens = length_function(state["collapsed_summaries"])

    if num_tokens > token_max:
        return "collapse_summaries"
    else:
        return "generate_final_summary"

# 为单个文档生成摘要（Map阶段的核心函数）
async def generate_summary(state: SummaryState):
    prompt = map_prompt.invoke(state["content"])
    response = await llm.ainvoke(prompt)
    return {"summaries": [response.content]}

# 收集摘要：将并行生成的摘要收集成文档列表
def collect_summaries(state: OverallState):
    return {"collapsed_summaries": [Document(summary) for summary in state["summaries"]]}

# 智能合并摘要：根据token限制分批合并摘要
async def collapse_summaries(state: OverallState):
    doc_lists = split_list_of_docs(
        state["collapsed_summaries"],
        length_function,
        token_max,
    )

    results = []
    for doc_list in doc_lists:
        results.append(await acollapse_docs(doc_list, _reduce))

    return {"collapsed_summaries": results}

# 生成最终摘要结果
async def generate_final_summary(state: OverallState):
    response = await _reduce(state["collapsed_summaries"])
    return {"final_summary": response}

graph = StateGraph(OverallState)
graph.add_node("generate_summary", generate_summary)
graph.add_node("collect_summaries", collect_summaries)
graph.add_node("collapse_summaries", collapse_summaries)
graph.add_node("generate_final_summary", generate_final_summary)

graph.add_conditional_edges(START, map_summaries, ["generate_summary"])
graph.add_edge("generate_summary", "collect_summaries")
graph.add_conditional_edges("collect_summaries", should_collapse)
graph.add_conditional_edges("collapse_summaries", should_collapse)
graph.add_edge("generate_final_summary", END)

app = graph.compile()

In [12]:
async for step in app.astream(
    {"contents": [doc.page_content for doc in split_docs]},
    {"recursion_limit": 10},
):
    print(list(step.keys()))
    if "generate_final_summary" in step:
        print(step["generate_final_summary"]["final_summary"])

['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['generate_summary']
['collect_summaries']
['collapse_summaries']
['collapse_summaries']
['collapse_summaries']
['collapse_summaries']
['collapse_summaries']
['generate_final_summary']
基于大型语言模型（LLM）的人工智能代理系统通过整合**规划、记忆和工具使用**三大核心能力，结合推理与行动（ReAct框架）及自我反思机制，实现了复杂任务的自主执行与持续改进，被视为迈向通用人工智能（AGI）的重要进展。

当前发展聚焦两大方向：
1. **增强通用模型能力**：通过集成外部工具（如API、专家模型）突破LLM固有限制，实现复杂任务自动化与专业应用，同时面临参数提取、安全风险等挑战；
2. **赋能专业开发流程**：通过AI驱动的代码开发工具集、结构化需求澄清和代码质量标准，系统化提升软件开发效率。

技术瓶颈主要体现在：
- 上下文窗口限制导致扩展困难
- 长期任务规划与错误恢复的稳健性不足
- 自然语言接口可靠性问题（格式错误、指令拒绝等）
- 对代码生成质量的高标准要求（需完整可执行且符合架构规范）

总体而言，LLM智能体在展现强大问题解决能力的同时，仍需在上下文处理、规划稳健性和接口可靠性等关键技术环节实现突破。
